In [ ]:
# Transfer ORCAS from MS Marco V1 to MS Marco V2

It is discouraged to use the ORCAS dataaset on MS Marco V2 (for training of models, since this could potentially leak data), but here we use the ORCAS dataset unsupervised for our entry-page finding experiments.

In [24]:
import ir_datasets
from tqdm import tqdm
import json

def normalize_url(url):
    url = url.lower().split('://')
    assert len(url) == 2
    
    return url[1]

def iterator_marco_v1():
    for i in ir_datasets.load('msmarco-document').docs_iter():
        yield {
            'url': normalize_url(i.url),
            'id': i.doc_id
        }


def iterator_marco_v2():
    for i in ir_datasets.load('msmarco-document-v2').docs_iter():
        yield {
            'url': normalize_url(i.url),
            'id': i.doc_id
        }
            

In [21]:
v1_id_to_url = {}
for i in tqdm(iterator_marco_v1()):
    v1_id_to_url[i['id']] = i['url']

3213835it [02:02, 26269.38it/s]


In [23]:
url_to_v2_id = {}
for i in tqdm(iterator_marco_v2()):
    url_to_v2_id[i['url']] = i['id']

11959635it [09:15, 21536.66it/s]


In [25]:
ORCAS_DIR = '/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-probst/retrievalExperiments/samples/orcas/'
matched = 0
unmatched = 0

with open(ORCAS_DIR + 'orcas-docs.jsonl', 'r') as src, open(ORCAS_DIR + 'orcas-marco-v2-docs.jsonl', 'w') as trg:
    for doc in tqdm(src):
        doc = json.loads(doc)
        target_url = v1_id_to_url[doc['id']]
        
        if target_url in url_to_v2_id:
            doc['id'] = url_to_v2_id[target_url]
            trg.write(json.dumps(doc) +'\n')
            matched += 1
        else:
            unmatched += 1

1422027it [00:52, 27106.37it/s]


In [26]:
print('Matched documents ', matched)
print('Unmatched documents ', unmatched)
print('Percentage matched ', matched/(matched + unmatched))

Matched documents  1291719
Unmatched documents  130308
Percentage matched  0.9083646091107975
